Copyright 2017 Amazon.com, Inc. or its affiliates. All Rights Reserved.<br /><br />Licensed under the Amazon Software License (the "License"). You may not<br />use this file except in compliance with the License. A copy of the<br />License is located at:<br />   http://aws.amazon.com/asl/<br />or in the "license" file accompanying this file. This file is distributed<br />on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, express<br />or implied. See the License for the specific language governing permissions<br />and limitations under the License.
# Launch a Tuning Job

In [ ]:
import smhpolib

In [ ]:
region = 'us-west-2'

In [ ]:
import os
os.environ["AWS_REGION"] = region

In [ ]:
# Create a launcher object for XGBoost
launcher = smhpolib.launcher.XGBoostLauncher()
# Look at the launcher -- see that it's not fully configured yet.
launcher  

**NOTE:** You must actually change these hyperparameter ranges, because these defaults won't work!

In [ ]:
# Configure the hyper-parameter ranges we'll search over
# Refer to https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
# Suggest trying eta, gamma, min_child_weight, max_depth, num_round
launcher.hyperparam_ranges = {
  "StaticHyperParameters": {
      "eval_metric": "auc",   # Don't change this
      "objective": "binary:logistic",   # Don't change this
      # Setting some fixed hyper-parameters
      "rate_drop": "0.3",
      "tweedie_variance_power": "1.4",
      "num_round": "100",
  },
  "ContinuousParameterRanges": [
    {
      "Type": "Continuous",
      "Name": "eta",
      "MinValue": "0",
      "MaxValue": "1"
    },
    {
      "Type": "Continuous",
      "Name": "gamma",
      "MinValue": "0",
      "MaxValue": "10"
    },
    {
      "Type": "Continuous",
      "Name": "min_child_weight",
      "MinValue": "1",
      "MaxValue": "10"
    }
  ],
  "CategoricalParameterRanges": [
  ],
  "IntegerParameterRanges": [
    {
      "Type": "Integer",
      "Name": "max_depth",
      "MinValue": "1",
      "MaxValue": "10"
    },
  ]
}

In [ ]:
# We need the account id to specify the names of the access role and S3 bucket
account_id = smhpolib.util.current_aws_account()
account_id

In [ ]:
# configure other aspects of the tuning job...

launcher.TRAINING_ROLE = "arn:aws:iam::%s:role/AWSSagemakerHPODataAccessRole" % account_id
launcher.opts.output_data_url = "s3://sagemaker-output-%s-%s/" % (account_id,region)
launcher.opts.input_data_url = "s3://public-test-hpo-datasets-%s/kaggle/porto-seguro/xgb/" % region

# How many total training jobs for the entire Tuning Job
launcher.opts.total_jobs = 10
# Degree of parallelism -- how many to do at once
launcher.opts.parallel_jobs = 3

In [ ]:
# check the launcher
launcher

In [ ]:
# Look at the CreateTuningJob request object we'll be sending
import json
req = launcher.create_tuning_job_request()
print(json.dumps(req, indent=2, sort_keys=True))

In [ ]:
# Launch the job!
response = launcher.launch_tuning_job()

_Did you get a ValidationError?  Check the hyper-parameter ranges. _

In [ ]:
tuning_job_arn = response['TuningJobArn']
tuning_job_name = tuning_job_arn[52:]  #Sorry this is ugly
tuning_job_name

In [ ]:
import time
time.sleep(5)  # describe-tuning-job is eventually consistent.

In [ ]:
smhpo = smhpolib.get_smhpo_client(region=region)
smhpo.describe_tuning_job(TuningJobName=tuning_job_name)